# API

In [81]:
import pandas as pd
import numpy as np
import boto3
import matplotlib.pyplot as plt
from os import walk
import os
import glob
from pathlib import Path
import io
import branca.colormap as cm

from shapely.geometry.polygon import Polygon
from shapely.geometry import box
from shapely.geometry import Polygon, LineString, Point
import shapely.wkt
import geopandas as gpd
from geopandas import GeoDataFrame

import folium
import matplotlib.pyplot as plt

import json
from folium.plugins import FastMarkerCluster
from folium import plugins

import skmob
from skmob.preprocessing import filtering
from skmob.preprocessing import detection

import datetime
from geopy.distance import great_circle

# import data

In [ ]:
# import mobilewalla data and create table
# name the table in the format m_[region name]_yyyymm
"""
create external table m_mobilewalla_location_hk(
ifa string,
ASN string,
Latitude double,
Longitude double,
bundle_id string,
Carrier string,
`Timestamp` timestamp,
device_category  string,
device_name string,
ip_address string,
major_os string,
City string,
connection_type string,
Country string,
state string,
Zip string,
Platform string,
store_url string,
user_agent string
)
STORED AS PARQUET
LOCATION 's3://alantuan/HK/'
tblproperties ("parquet.compression"="SNAPPY")
"""

In [ ]:
# calculate daily distance and other features
"""
SELECT ifa, date(timestamp) day, avg(AvgLat) avg_lat, avg(AvgLon) avg_lon,
AVG(great_circle_distance(latitude,longitude,AvgLat,AvgLon)) as avg_dist_radius,
stddev_pop(great_circle_distance(latitude,longitude,AvgLat,AvgLon)) as std_dist_radius,
MAX(great_circle_distance(latitude,longitude,AvgLat,AvgLon)) as max_radius,
MIN(great_circle_distance(latitude,longitude,AvgLat,AvgLon)) as min_radius,
count(distinct cast(latitude as varchar) || '_' || cast(longitude as varchar)) num_location,
count(distinct asn) num_app, array_agg(distinct asn) app_list
FROM "default"."m_location_sample_hk_intmdclean" JOIN(SELECT ifa as ui, date(timestamp)  AS d,
AVG(latitude) AS AvgLat, AVG(longitude) AS AvgLon FROM "default"."m_location_sample_hk_intmdclean"
GROUP BY date(timestamp) ,ifa) t2
ON "default"."m_location_sample_hk_intmdclean".ifa = t2.ui AND date(timestamp)=t2.d 
GROUP by  date(timestamp) , ifa
ORDER BY date(timestamp) 
limit 100
"""
# need to add total distance

# Directly access s3 data

In [11]:
with open('../aws1.txt') as f:
    lines = f.readlines()
    

In [12]:
s3 = boto3.client('s3',region_name='us-east-1',
    aws_access_key_id = lines[1].strip(' \n'),
    aws_secret_access_key= lines[4])
obj = s3.get_object(Bucket = "alantuan",Key='hk/dt=20210620/part-00008-bbe04846-1c93-4d94-8070-617ad2e3ec8c-c000.snappy.parquet')
hk = pd.read_parquet(io.BytesIO(obj['Body'].read()))

In [13]:
hk.columns

Index(['ifa', 'asn', 'latitude', 'longitude', 'bundle_id', 'carrier',
       'timestamp', 'device_category', 'device_name', 'ip_address', 'major_os',
       'city', 'connection_type', 'country', 'state', 'zip', 'platform',
       'store_url', 'user_agent'],
      dtype='object')

In [335]:
s3 = boto3.client('s3',region_name='us-east-1',
    aws_access_key_id = lines[1].strip(' \n'),
    aws_secret_access_key= lines[4])
obj = s3.get_object(Bucket = "alantuan",Key='aluo_hk_yingkwai/aluo_202201_1stweek_resident5/20220124_043518_00009_2sdid_fddce3d2-c285-418f-a808-aa431ae4ed37')


NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [8]:
# get all the data from a folder

## get list of file names
s3 = boto3.resource('s3',region_name='us-east-1',
    aws_access_key_id = lines[1].strip(' \n'),
    aws_secret_access_key= lines[4])
my_bucket = s3.Bucket('alantuan')
l = [i.key for i in my_bucket.objects.filter(Prefix="aluo_hk_yingkwai/aluo_220101_resident5/")] 

## get the data
s3_client = boto3.client('s3',region_name='us-east-1',
    aws_access_key_id = lines[1].strip(' \n'),
    aws_secret_access_key= lines[4])
li = []
for i in l:
    obj = s3_client.get_object(Bucket = "alantuan",Key=i)
    li.append(pd.read_parquet(io.BytesIO(obj['Body'].read())))

## combine into one
hk = pd.concat(li, axis=0, ignore_index=True)


In [9]:
hk

,ifa,asn,latitude,longitude,bundle_id,carrier,timestamp,device_category,device_name,ip_address,major_os,city,connection_type,country,state,zip,platform,store_url,user_agent
0,74ec85a9-b31c-4718-a342-a4b703186ada,,22.282593,114.153575,,Hutchison Telephone Company Limited,2022-01-18 00:10:01,,,124.217.188.94,,Hong Kong,CELLULAR,HK,HK,,ANDROID,,9
1,74ec85a9-b31c-4718-a342-a4b703186ada,,22.369086,114.126217,,Netvigator,2022-01-18 13:32:07,,,119.236.19.36,,Hong Kong,WIFI,HK,HK,,ANDROID,,9
2,74ec85a9-b31c-4718-a342-a4b703186ada,,22.340912,114.154702,,Hutchison Telephone Company Limited,2022-01-18 04:20:50,,,124.217.188.94,,Hong Kong,CELLULAR,HK,HK,,ANDROID,,9
3,74ec85a9-b31c-4718-a342-a4b703186ada,,22.340720,114.152362,,Hutchison Telephone Company Limited,2022-01-18 04:29:05,,,124.217.188.94,,Hong Kong,CELLULAR,HK,HK,,ANDROID,,9
4,74ec85a9-b31c-4718-a342-a4b703186ada,,22.368839,114.125970,,Hutchison Telephone Company Limited,2022-01-18 09:40:29,,,124.217.188.94,,Hong Kong,CELLULAR,HK,HK,,ANDROID,,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,e4f1341a-5b92-487b-8b7e-4941caa90215,,22.369372,114.126173,,Netvigator,2022-01-22 07:11:53,,,42.2.142.159,,Hong Kong,WIFI,HK,HK,,ANDROID,,7.0
7028,e4f1341a-5b92-487b-8b7e-4941caa90215,,22.369401,114.126078,,Netvigator,2022-01-22 03:49:57,,,42.2.142.159,,Hong Kong,WIFI,HK,HK,,ANDROID,,7.0
7029,e4f1341a-5b92-487b-8b7e-4941caa90215,,22.369389,114.126258,,Netvigator,2022-01-22 10:54:35,,,42.2.142.159,,Hong Kong,WIFI,HK,HK,,ANDROID,,7.0
7030,e4f1341a-5b92-487b-8b7e-4941caa90215,,22.369393,114.126096,,Netvigator,2022-01-22 06:04:09,,,42.2.142.159,,Hong Kong,WIFI,HK,HK,,ANDROID,,7.0


# Daily Districs Movement - FLOWMAP.BLUE 

## all data without any modification

In [ ]:
"""
create table m_hk_district_movement_v1 as
with df1 as (
select a.*, b.name as hk_district, b.geometry from m_hk a cross join m_hk_districts b
where ST_Contains(ST_Polygon(b.geometry), ST_Point(a.longitude, a.latitude))
),

df2 as(
select *, LAG(hk_district) OVER (PARTITION BY ifa, date(timestamp) ORDER BY timestamp) as previous_district
from df1),

df3 as (
select * from df2 
where previous_district is not NULL
)

select date(timestamp) date, hk_district as dest, previous_district as origin, count(ifa) as count from df3
group by date(timestamp), hk_district, previous_district


"""

### convert to wide dataframe

In [352]:
# save the original long data into wide daily matrix (each day is a file)
df = pd.read_csv('../data/0_m_hk_district_movement_v1.csv')
df = df.sort_values('date').reset_index(drop=True)
days = df['date'].unique()

for i in days:
    df1 = df.loc[df['date']==i]
    df1 = df1.pivot(index='origin',columns='dest', values='count')
    df1.to_csv(f"../outputs/0_hk_district_movement/m_hk_district_movement_{''.join(i.split('-'))}.csv")

In [345]:
df1

dest,Central and Western,Eastern,Islands,Kowloon City,Kwai Tsing,Kwun Tong,North,Sai Kung,Sha Tin,Sham Shui Po,Southern,Tai Po,Tsuen Wan,Tuen Mun,Wan Chai,Wong Tai Sin,Yau Tsim Mong,Yuen Long
origin,,,,,,,,,,,,,,,,,,
Central and Western,1601542,1909,898,1407,972,2447,716,1553,1328,909,1134,472,850,1297,4104,1117,3041,1614
Eastern,1891,1378564,204,489,136,1344,58,550,293,292,450,52,92,78,2840,206,1089,141
Islands,885,204,604713,285,452,476,124,129,551,209,262,360,869,295,232,173,392,250
Kowloon City,1363,487,252,1577028,687,2724,506,916,1704,1506,222,431,337,393,925,2330,5571,857
Kwai Tsing,952,150,470,642,1457130,628,66,129,865,1768,54,151,3281,365,195,350,975,418
Kwun Tong,2510,1404,449,2802,607,2325587,314,2617,1283,982,335,432,333,498,1225,2109,1884,1254
North,727,50,125,513,85,307,1030162,102,921,153,56,1692,121,147,83,88,291,825
Sai Kung,1519,553,136,904,148,2592,99,1371027,681,183,101,202,128,52,523,691,470,179
Sha Tin,1311,304,534,1710,902,1321,869,674,2151201,898,122,2372,473,239,604,701,1418,377


## district movement with distinct ifa for each day

In [ ]:
"""
create table m_hk_district_movement_v2 as
with df1 as (
select a.*, b.name as hk_district, b.geometry from m_hk a cross join m_hk_districts b
where ST_Contains(ST_Polygon(b.geometry), ST_Point(a.longitude, a.latitude))
),

df2 as(
select *, LAG(hk_district) OVER (PARTITION BY ifa, date(timestamp) ORDER BY timestamp) as previous_district
from df1),

df3 as (
select * from df2 
where previous_district is not NULL
)

select date(timestamp) date, hk_district as dest, previous_district as origin, count(distinct ifa) as count from df3
group by date(timestamp), hk_district, previous_district

"""

In [354]:
# save the original long data into wide daily matrix (each day is a file)
df = pd.read_csv('../data/0_m_hk_district_movement_v2.csv')
df = df.sort_values('date').reset_index(drop=True)
days = df['date'].unique()

for i in days:
    df1 = df.loc[df['date']==i]
    df1 = df1.pivot(index='origin',columns='dest', values='count')
    df1.to_csv(f"../outputs/0_hk_district_movement_v2/m_hk_district_movement_v2_{''.join(i.split('-'))}.csv")